**Employee Importance**

You are given a data structure of employee information, which includes the employee's unique id, his importance value and his direct subordinates' id.

For example, employee 1 is the leader of employee 2, and employee 2 is the leader of employee 3. They have importance value 15, 10 and 5, respectively. Then employee 1 has a data structure like [1, 15, [2]], and employee 2 has [2, 10, [3]], and employee 3 has [3, 5, []]. Note that although employee 3 is also a subordinate of employee 1, the relationship is not direct.

Now given the employee information of a company, and an employee id, you need to return the total importance value of this employee and all his subordinates.

Example 1:
```
Input: [[1, 5, [2, 3]], [2, 3, []], [3, 3, []]], 1
Output: 11
Explanation:
Employee 1 has importance value 5, and he has two direct subordinates: employee 2 and employee 3. They both have importance value 3. So the total importance value of employee 1 is 5 + 3 + 3 = 11.
```
Ok so first thing's first, we need a better way to access each employee than looping through and looking for the right id each time. so we should set up a dictionary of employee.id : employee for each employee in employees. This set up will take O(n) but enable O(1) look up afterward. 

Then we can do a simple search, either bfs or dfs, while keeping track of the total importance of each employee that ends up enqueued/stacked. 

The runtime will be the number of subordinates of the target employee. Because there's no way to tell how many that will be, the general runtime will be O(n) where n = number of total employees. Same goes for the space complexity of the queue. 

In [ ]:
"""
# Employee info
class Employee:
    def __init__(self, id: int, importance: int, subordinates: List[int]):
        # It's the unique id of each node.
        # unique id of this employee
        self.id = id
        # the importance value of this employee
        self.importance = importance
        # the id of direct subordinates
        self.subordinates = subordinates
"""
class Solution:
    def getImportance(self, employees: List['Employee'], id: int) -> int:
        empMap = {emp.id: emp for emp in employees}
        if id not in empMap: return
        target = empMap[id]
        total = 0
        q = [target]
        while q:
            curr = q.pop(0)
            total += curr.importance
            for sub in curr.subordinates:
                q.append(empMap[sub])
        return total

**Oranges Rotting**

In a given grid, each cell can have one of three values:

- the value 0 representing an empty cell;
- the value 1 representing a fresh orange;
- the value 2 representing a rotten orange.
- Every minute, any fresh orange that is adjacent (4-directionally) to a rotten orange becomes rotten.

Return the minimum number of minutes that must elapse until no cell has a fresh orange.  If this is impossible, return -1 instead.

Hmmm. This seems like almost an inverse bfs, because we want to track all the fresh oranges rather than the rotten ones, but the rotten ones are behaving like a bfs. Every minute, we need to go through all of the fresh oranges and check if they've been rotted. But we can't immediately update them because that will affect the other oranges too soon. so at each minute, we need to loop through all the fresh oranges, track which oranges to rot, then rot them. Once they've rotted, we can remove them from the list of fresh oranges to check. Once there are no more oranges to check, we can return the number of minutes it's been. If at any point while we're looping, no new oranges have been added to rot but there are more to check, we know that we can't rot all the oranges and can return -1. So we just need to figure out how to loop through the grid with guards against exceeding the length of the lists. 

The runtime on this is long. Suppose n is the number of squares in the grid. first we have a O(n) to populate `toCheck`. but it's an addition to the rest, not a multiplication. Then at each 'minute', we loop through all the squares with 1s in them. We check the 4 squares directly around them, but that's a constant. This process gets shorter each time through but not in a consistent way, so this is also O(n). Then we loop through all the ones that have rotted, but that amount is affected directly by the size of `toCheck`, so this basically completes the O(n) of looping through `toCheck`. It's weird, we're checking more oranges than we have to each time because we should only be checking the ones touching rotting oranges, but leetcode says this answer is faster than 93% of the other submissions. But say the grid is one row with one rotten orange at the start and 1000 fresh ones. we'll have to go through all the fresh ones 1000 times, even as the inner loop gets smaller by one each time. Pretty sure that's O(n^2) worst case...

In [ ]:
class Solution:
    def orangesRotting(self, grid: List[List[int]]) -> int:
        mins = 0
        n = len(grid)
        m = len(grid[0])
        toCheck = set()
        for x in range(n):
            for y in range(m):
                if grid[x][y] == 1:
                    toCheck.add((x,y))
        dirs = [(1,0), (-1,0), (0,1), (0,-1)]
        while len(toCheck) > 0:
            toChange = set()
            for x, y in toCheck:
                for xD, yD in dirs:
                    if 0 <= x + xD < n and 0 <= y + yD < m and grid[x+xD][y+yD] == 2:
                        toChange.add((x,y))
                        break
            if len(toChange) == 0: return -1
            for x, y in toChange:
                grid[x][y] = 2
            toCheck = toCheck - toChange
            mins += 1
        return mins

**Construct Binary Tree from Inorder and Postorder Traversal**

Given inorder and postorder traversal of a tree, construct the binary tree.

Note:
You may assume that duplicates do not exist in the tree.

For example, given

inorder = [9,3,15,20,7]
postorder = [9,15,7,20,3]
Return the following binary tree:

```
    3
   / \
  9  20
    /  \
   15   7
```

So the puzzle here is to figure out how to find the order of the nodes. neither traversal gives you all the info you need on its own. So where to start? Postorder requires that for every parent node, all nodes below that parent (child, grandchild, etc) appear before it. This means that the root of the tree will always be the last item in the list because it is the initial parent. you can then take that value and know in the inorder traversal that everything to the left of the root value (this problem requires that all values in the tree are unique) is the left subtree of the root, and everything to the right is its right subtree. you can then pop the root off of the postorder listand break the inorder list around the root so that you don't ever look at it twice. then to find the root of each subtree, you repeat the process. you have to start with the right subtree every time, because the postorder traversal was built from left to right, so building it back up requires that you start from the back, ie the right. the root's right child must be set to whatever the function returns recursively when called on the updated traversals (removing the current root) and the root itself. 

Worst case, this solution is actually O(n^2). this is because for each value from the postorder list, you need to find its index in the inorder list. this is a O(n) procedure, occuring on each of n nodes. if the entire tree is a single branch off to the left, it will need to loop through all of the lower level nodes every time it creates a new node. The average runtime, however, is much better. We are also not using any space other than the recursive stack and the nodes we create. we only actually recurse n times, and there are n nodes (additional, not multiplicative), so the space complexity is O(n). 

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution:
    def buildTree(self, inorder: List[int], postorder: List[int]) -> TreeNode:
        if not inorder: return 
        root = postorder.pop(-1)
        return self.constructBranch(root, inorder, postorder)
  
    def constructBranch(self, root, inorder, postorder):
        rootNode = TreeNode(root)
        inLen = len(inorder)
        if inLen == 0 or inLen == 1: return rootNode
        inRootI = inorder.index(root)
        if inRootI != inLen - 1:
            inR = inorder[inRootI + 1:]
            root = postorder.pop(-1)
            rootNode.right = self.constructBranch(root, inR, postorder)
        if inRootI != 0:
            inL = inorder[:inRootI]
            root = postorder.pop(-1)
            rootNode.left = self.constructBranch(root, inL, postorder)
        return rootNode